# Predictions and evaluation
* Load model, predict and store masks
* Compute IoU metric
* Visualize examples (high/low IoU per class; image + ground truth mask + prediction)

## Select which models to load (user input only required here)
Type in which model from the `saved_model` folder to load for which cell type. If the same model is used to predict the masks for each cell type, type in the same model name for all cell types.

In [ ]:
mdl_astro  = "shsy5y_MobileNetV2_steps_12_epochs_2+2" # User input: specify model here
mdl_cort   = "shsy5y_MobileNetV2_steps_12_epochs_2+2" # User input: specify model here
mdl_shsy5y = "shsy5y_MobileNetV2_steps_12_epochs_2+2" # User input: specify model here

mdl_dict = {"astro": mdl_astro, "cort": mdl_cort, "shsy5y": mdl_shsy5y}

## Imports

In [ ]:
import os
import pathlib
import shutil
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.transform import resize
import image_modeling_import   # import image_modeling_import.py file

In [ ]:
# Get the directory of the repository
curr_dir = os.getcwd()
parent_dir = pathlib.Path(curr_dir).parents[1]

preproc_path = f"{parent_dir}/data/data_preprocessed/"

## Specify folder name for predictions

## Predict, resize and store the masks based on the sliced images

In [ ]:
# Read one example image to get original size
df = pd.read_csv("CSVs/cells_train_astro.csv", header = None)
img = imageio.imread(df.loc[0,0])
original_size = img.shape

In [ ]:
for cell_type, model_name in mdl_dict.items():
        # Get image directories and model based on model selection above
        TRAIN_PATH = f"CSVs/cells_train_{cell_type}.csv"
        TEST_PATH  = f"CSVs/cells_test_{cell_type}.csv"
        EVAL_PATH  = f"CSVs/cells_eval_{cell_type}.csv"
        img_paths  = [TRAIN_PATH, EVAL_PATH, TEST_PATH]
        
        trained_model = tf.keras.models.load_model(f'saved_model/{model_name}')
        
        for path in img_paths:
            # Load images as tensorflow dataset
            dataset = image_modeling_import.load_dataset_seg(path, 10, training=False)
            
            # Load metadata to get image ID
            df = pd.read_csv(path, header = None)
            df["id"] = df.apply(lambda x: x.iloc[0].split("/")[-1].split(".")[0], axis = 1)
            
            # Predict masks and convert predictions in single channel numpy arrays that can be visualized as image
            predictions = trained_model.predict(dataset)
            predictions = tf.argmax(predictions, axis=-1)
            predictions = predictions[..., tf.newaxis]
            predictions = predictions.numpy()
            predictions = predictions
            
            # Resize and store
            for i, pred in enumerate(predictions):
                pred_resized = resize(pred, original_size, anti_aliasing = False, preserve_range = True).astype(dtype = np.uint8)
                imageio.imwrite(f"{preproc_path}sliced_images/predictions/masks/{df['id'][i]}_pred.png", pred_resized)
        

## Recombine sliced predictions

In [ ]:
# Since we already made the per-cell-type-model predictions
# and the train/test/eval split is identical for all-cell-types models and per-cell-type models,
# we can use the csv files that are not additionally split per cell type here.
TRAIN_PATH = 'CSVs/cells_train.csv'
EVAL_PATH  = 'CSVs/cells_eval.csv'
TEST_PATH  = 'CSVs/cells_test.csv'

In [ ]:
# Instantiate lists that are filled with the desired output dataframe values (concatenate df after the loop below)
img_id = list()
data_subset = list()
IoU = list()

img_paths = {"train": TRAIN_PATH, "eval": EVAL_PATH, "test": TEST_PATH}


for ds, path in img_paths.items():
    # Load metadata to get image ID
    df = pd.read_csv(path, header = None)
    df["id"] = df.apply(lambda x: x.iloc[0].split("/")[-1].split(".")[0], axis = 1)
    
    # ID without slicing suffix
    id_combined_list = df["id"].apply(lambda x: x.split("_")[0]).unique().tolist()
    
    for id_comb in id_combined_list:
        
        # Recombine the sliced masks
        img_a = imageio.imread(f"{preproc_path}sliced_images/predictions/masks/{id_comb}_a_pred.png")
        img_b = imageio.imread(f"{preproc_path}sliced_images/predictions/masks/{id_comb}_b_pred.png")
        img_c = imageio.imread(f"{preproc_path}sliced_images/predictions/masks/{id_comb}_c_pred.png")
        img_d = imageio.imread(f"{preproc_path}sliced_images/predictions/masks/{id_comb}_d_pred.png")
        
        img_left  = np.concatenate((img_a, img_c))
        img_right = np.concatenate((img_b, img_d))
        
        mask_pred = np.concatenate((img_left, img_right), axis = 1).astype(dtype = np.uint8)
        
        imageio.imwrite(preproc_path + "mask_predicted/masks/" + id_comb + "_pred.png", mask_pred)
        
        # Compute and store intersections, unions, and IoU compared to original masks
        # mask_true = imageio.imread(preproc_path + "mask_groundtruth/" + id_comb + "_mask.png") # use original masks
        mask_true = imageio.imread(preproc_path + "masks_used_for_model/" + id_comb + "_mask.png") # use masks specified in 04_unet_data_pipeline
        
        intersection = ((mask_pred!=0) & (mask_true!=0)).astype(dtype = np.uint8)
        union        = ((mask_pred!=0) | (mask_true!=0)).astype(dtype = np.uint8)
        
        imageio.imwrite(preproc_path + "mask_predicted/intersections/" + id_comb + "_inter.png", intersection)
        imageio.imwrite(preproc_path + "mask_predicted/unions/" + id_comb + "_union.png", union)
        
        
        IoU.append(np.sum(intersection)/np.sum(union))
        
        img_id.append(id_comb)
        data_subset.append(ds)
        

        
df_IoU = pd.DataFrame({"dataset": data_subset, "id": img_id, "IoU": IoU})

In [ ]:
df_IoU = pd.DataFrame({"dataset": data_subset, "id": img_id, "IoU": IoU})
df_IoU.head(10)

In [ ]:
# Match the cell type and store 
df = pd.read_csv("~/neuefische/ds-capstone-img-classification-segmentation/data/data_original/train.csv") # Original train.csv from Kaggle
df.drop_duplicates(subset = ["id"], inplace = True)

df_IoU = df_IoU.merge(df[["id","cell_type"]], on = "id", how = "left")

df_IoU.to_csv("IoU.csv", index = False)

df_IoU.head(10)

In [ ]:
df_IoU.to_csv("IoU.csv", index = False)

## Copy predictions to new, model-specific folder
...to avoid overwriting with predictions of different model

### Folder name based on model type, number of epochs, number of steps per epoch

In [ ]:
if mdl_dict["astro"]==mdl_dict["shsy5y"]==mdl_dict["cort"]:
    n_steps  = int(mdl_dict["astro"].split("_")[3])

    n_epochs = mdl_dict["astro"].split("_")[-1]
    n_epochs = n_epochs.split("+")
    n_epochs = str(np.sum([int(ep) for ep in n_epochs]))
    
    pred_folder_name = mdl_dict["astro"].split("_")[1]
    pred_folder_name = f"{pred_folder_name}_allS{n_steps}E{n_epochs}"
    
else:
    pred_folder_name = mdl_dict["astro"].split("_")[1]
    
    for celltype, model in mdl_dict.items():
        
        n_steps = model.split("_")[-3]
        
        n_epochs = model.split("_")[-1]
        n_epochs = n_epochs.split("+")
        n_epochs = str(np.sum([int(ep) for ep in n_epochs]))
        
        cell_name = celltype
        cell_name = "".join([cell_name, 'S'])
        cell_name = "".join([cell_name, n_steps])
        cell_name = "".join([cell_name, 'E'])
        cell_name = "".join([cell_name, n_epochs])
        
        pred_folder_name = "_".join([pred_folder_name, cell_name])
    
print(pred_folder_name)

### Copy predicted masks, intersections and unions in new folder

In [ ]:
for folder in ["intersections", "masks", "unions"]:
    
    dir_source = f"{preproc_path}mask_predicted/{folder}"
    dir_target = f"{preproc_path}mask_predicted/{pred_folder_name}/{folder}"

    shutil.copytree(dir_source, dir_target, dirs_exist_ok=True)

### Copy IoU.csv in new, model-specific folder

In [ ]:
source = "IoU.csv"
dir_target = f"{preproc_path}mask_predicted/{pred_folder_name}"

shutil.copy(source, dir_target)

## Check with example images

In [ ]:
img_id = df_IoU["id"].sample(n=1).values[0]
img_id

In [ ]:
img_id = "0140b3c8f445"

In [ ]:
img = imageio.imread(f"{parent_dir}/data/data_original/train/{img_id}.png")
plt.imshow(img)

In [ ]:
msk = imageio.imread(preproc_path + "masks_used_for_model/{}_mask.png".format(img_id))
plt.imshow(msk)

In [ ]:
prd = imageio.imread(f"{preproc_path}/mask_predicted/masks/{img_id}_pred.png")
plt.imshow(prd)

In [ ]:
isn = imageio.imread(f"{preproc_path}/mask_predicted/intersections/{img_id}_inter.png")
plt.imshow(isn)

In [ ]:
uni = imageio.imread(f"{preproc_path}/mask_predicted/unions/{img_id}_union.png")
plt.imshow(uni)